In [186]:
import glob
import re
from collections.abc import Generator
from pathlib import Path
from typing import Any

import nd2
import numpy as np
from fractal_converters_tools.tile import Point, Tile, Vector
from fractal_converters_tools.tiled_image import (
    PlatePathBuilder,
    SimplePathBuilder,
    TiledImage,
)
from ngio import PixelSize, create_ome_zarr_from_array


In [265]:
dir = r"Z:\zmbstaff\9987\Raw_Data\BVC_converters\testData_NikonSD\ND_Acquisitions"
dir = r"Z:\zmbstaff\9987\Raw_Data\BVC_converters\testData_Nikon_ti2"
fns = glob.glob(dir + "\\*.nd2")

In [266]:
fns

['Z:\\zmbstaff\\9987\\Raw_Data\\BVC_converters\\testData_Nikon_ti2\\one channel.nd2',
 'Z:\\zmbstaff\\9987\\Raw_Data\\BVC_converters\\testData_Nikon_ti2\\one channel001.nd2',
 'Z:\\zmbstaff\\9987\\Raw_Data\\BVC_converters\\testData_Nikon_ti2\\one channel002.nd2',
 'Z:\\zmbstaff\\9987\\Raw_Data\\BVC_converters\\testData_Nikon_ti2\\one channel003.nd2',
 'Z:\\zmbstaff\\9987\\Raw_Data\\BVC_converters\\testData_Nikon_ti2\\one channel004.nd2',
 'Z:\\zmbstaff\\9987\\Raw_Data\\BVC_converters\\testData_Nikon_ti2\\one channel005.nd2',
 'Z:\\zmbstaff\\9987\\Raw_Data\\BVC_converters\\testData_Nikon_ti2\\one channel006.nd2']

In [267]:
for fn in fns:
    data = nd2.imread(fn, xarray=True, dask=True)
    print(fn)
    print(data.sizes)

Z:\zmbstaff\9987\Raw_Data\BVC_converters\testData_Nikon_ti2\one channel.nd2
Frozen({'Y': 2304, 'X': 2304})
Z:\zmbstaff\9987\Raw_Data\BVC_converters\testData_Nikon_ti2\one channel001.nd2
Frozen({'C': 3, 'Y': 2304, 'X': 2304})
Z:\zmbstaff\9987\Raw_Data\BVC_converters\testData_Nikon_ti2\one channel002.nd2
Frozen({'C': 3, 'Y': 4378, 'X': 4378})
Z:\zmbstaff\9987\Raw_Data\BVC_converters\testData_Nikon_ti2\one channel003.nd2
Frozen({'C': 3, 'Y': 4378, 'X': 4378})
Z:\zmbstaff\9987\Raw_Data\BVC_converters\testData_Nikon_ti2\one channel004.nd2
Frozen({'Z': 9, 'C': 3, 'Y': 4378, 'X': 4378})
Z:\zmbstaff\9987\Raw_Data\BVC_converters\testData_Nikon_ti2\one channel005.nd2
Frozen({'Z': 9, 'C': 3, 'Y': 2304, 'X': 2304})
Z:\zmbstaff\9987\Raw_Data\BVC_converters\testData_Nikon_ti2\one channel006.nd2
Frozen({'T': 5, 'Z': 9, 'C': 3, 'Y': 2304, 'X': 2304})


In [276]:
fn = fns[0]
fn

'Z:\\zmbstaff\\9987\\Raw_Data\\BVC_converters\\testData_Nikon_ti2\\one channel.nd2'

In [269]:
fn = r"Z:\zmbstaff\9987\Raw_Data\BVC_converters\testData_NikonSD\JOBS_WellPlate\20250506_124144_018\WellB02_ChannelSD DAPI- EM,SD GFP - EM_Seq0000.nd2"
# fn = r"Z:\zmbstaff\9987\Raw_Data\BVC_converters\testData_NikonSD\JOBS_WellPlate\20250506_123741_693\WellB02_PointB02_0000_ChannelSD DAPI- EM,SD GFP - EM_Seq0000.nd2"

In [277]:
data = nd2.imread(fn, xarray=True, dask=True)

In [278]:
data

<xarray.DataArray '_dask_block-fd7dadaed45a8434979650a23ff10a09' (Y: 2304,
                                                                  X: 2304)> Size: 11MB
dask.array<_dask_block, shape=(2304, 2304), dtype=uint16, chunksize=(2304, 2304), chunktype=numpy.ndarray>
Coordinates:
  * Y        (Y) float64 18kB 0.0 0.3239 0.6477 0.9716 ... 745.2 745.5 745.8
  * X        (X) float64 18kB 0.0 0.3239 0.6477 0.9716 ... 745.2 745.5 745.8
Attributes:
    metadata:  {'metadata': Metadata(contents=Contents(channelCount=1, frameC...

In [279]:
data.attrs["metadata"].keys()

dict_keys(['metadata', 'experiment', 'attributes', 'text_info'])

In [280]:
data.attrs["metadata"]["metadata"]

Metadata(contents=Contents(channelCount=1, frameCount=1), channels=[Channel(channel=ChannelMeta(name='Ex395', index=0, color=Color(r=0, g=255, b=255, a=1.0), emissionLambdaNm=432.0, excitationLambdaNm=395.0), loops=None, microscope=Microscope(objectiveMagnification=20.0, objectiveName='Plan Apo λ 20x Ph2 DM', objectiveNumericalAperture=0.75, zoomMagnification=1.0, immersionRefractiveIndex=1.0, projectiveMagnification=None, pinholeDiameterUm=None, modalityFlags=['fluorescence', 'camera']), volume=Volume(axesCalibrated=(True, True, False), axesCalibration=(0.323859905725534, 0.323859905725534, 1.0), axesInterpretation=('distance', 'distance', 'distance'), bitsPerComponentInMemory=16, bitsPerComponentSignificant=16, cameraTransformationMatrix=(-0.99893357012953, 0.046170580116253024, -0.046170580116253024, -0.99893357012953), componentCount=1, componentDataType='unsigned', voxelCount=(2304, 2304, 1), componentMaxima=[0.0], componentMinima=[0.0], pixelToStageTransformationMatrix=None))])

In [281]:
data.attrs["metadata"]["metadata"].channels[0].volume

Volume(axesCalibrated=(True, True, False), axesCalibration=(0.323859905725534, 0.323859905725534, 1.0), axesInterpretation=('distance', 'distance', 'distance'), bitsPerComponentInMemory=16, bitsPerComponentSignificant=16, cameraTransformationMatrix=(-0.99893357012953, 0.046170580116253024, -0.046170580116253024, -0.99893357012953), componentCount=1, componentDataType='unsigned', voxelCount=(2304, 2304, 1), componentMaxima=[0.0], componentMinima=[0.0], pixelToStageTransformationMatrix=None)

In [282]:
data.attrs["metadata"]["experiment"]

[]

In [283]:
data.attrs["metadata"]["attributes"]

Attributes(bitsPerComponentInMemory=16, bitsPerComponentSignificant=16, componentCount=1, heightPx=2304, pixelDataType='unsigned', sequenceCount=1, widthBytes=4608, widthPx=2304, compressionLevel=None, compressionType=None, tileHeightPx=None, tileWidthPx=None, channelCount=1)

In [284]:
data.attrs["metadata"]["text_info"]

{'description': 'Metadata:\r\nDimensions: λ(1)\r\nCamera Name: Fusion, SN:001346\r\nNumerical Aperture: 0.75\r\nRefractive Index: 1\r\n Name: Ex395\r\n Component Count: 1\r\n Modality: Widefield Fluorescence\r\n Camera Settings:   Exposure: 100 ms\r\n  Binning: 1x1\r\n  Scan Mode: Fast\r\n  Temperature: 35.0°C\r\n  Denoise.ai OFF\r\n  Clarify.ai OFF\r\n Microscope Settings:   Microscope: Ti2 Microscope\r\n  Nikon Ti2, FilterChanger(Turret-Lo): 2 (MXR00244 - LED-DA/FI/TR/Cy5-4X-B (DAPI / FITC / TRITC / Cy5 - Pinkel Quad))\r\n  Nikon Ti2, FilterChanger(EM Wheel1): 2 (MXR00245 - LED-DA/FI/TR/Cy5-0X4M-B (DAPI / FITC / TRITC / Cy5 - Sedat Quad Emitters) (FF01-432/36) (Em1))\r\n  Nikon Ti2, Shutter(FL-Lo): Open\r\n  LightPath: L100\r\n  Condenser: 5 ()\r\n  PFS, mirror: Inserted\r\n  Zoom: 1.00x\r\n  Spectra/AuraII, Shutter(Spectra): Active\r\n  Spectra/AuraII, MultiLaser(Spectra):\r\n     Line:1; ExW:395; Power: 20.0; On',
 'capturing': 'Fusion, SN:001346\r\nExposure: 100 ms\r\nBinning: 1x1

In [285]:
nd2file.close()

AttributeError: 'NoneType' object has no attribute 'close'

In [159]:
nd2file = nd2.ND2File(fn)

In [160]:
nd2file.frame_metadata(0).channels[0].position.stagePositionUm.x

38848.6

In [161]:
nd2file.frame_metadata(0).position

AttributeError: 'FrameMetadata' object has no attribute 'position'

In [162]:
set(data.dims).issubset(("P", "Z", "Y", "X"))


False

In [163]:
data.dims

('C', 'Y', 'X')

In [286]:
class nd2TileLoader:
    """nd2 tile loader."""

    def __init__(self, path: str, p: int):
        """Initialize LifTileLoader."""
        self.path = path
        self.p = p

    @property
    def dtype(self):
        """Get the data type of the tile."""
        with nd2.ND2File(self.path) as nd2file:
            dtype = nd2file.dtype
        return dtype

    def load(self) -> np.ndarray:
        """Load the tile data."""
        tile_data = nd2.imread(self.path, xarray=True, dask=True)
        if "P" in tile_data.dims:
            tile_data = tile_data.isel(P=self.p)
        if not set(tile_data.dims).issubset(("T", "C", "Z", "Y", "X")):
            raise ValueError(
                f"Data can only have dimensions T, C, Z, Y, X. Found: {tile_data.dims}"
            )
        if "Z" not in tile_data.dims:
            tile_data = tile_data.expand_dims(Z=1, axis=0)
        if "C" not in tile_data.dims:
            tile_data = tile_data.expand_dims(C=1, axis=0)
        if "T" not in tile_data.dims:
            tile_data = tile_data.expand_dims(T=1, axis=0)
        if tile_data.dims != ("T", "C", "Z", "Y", "X"):
            tile_data = tile_data.transpose("T", "C", "Z", "Y", "X")

        return tile_data.data.compute()

In [287]:
def build_tiles(nd2file) -> Generator[Tile, Any, None]:
    """Build tiles from nd2 file."""
    shape_x = nd2file.sizes["X"] if "X" in nd2file.sizes else 1
    shape_y = nd2file.sizes["Y"] if "Y" in nd2file.sizes else 1
    shape_z = nd2file.sizes["Z"] if "Z" in nd2file.sizes else 1
    shape_c = nd2file.sizes["C"] if "C" in nd2file.sizes else 1
    shape_t = nd2file.sizes["T"] if "T" in nd2file.sizes else 1

    # scale factors [um]/[px]
    scale_x = nd2file.voxel_size().x
    scale_y = nd2file.voxel_size().y
    scale_z = nd2file.voxel_size().z
    scale_t = 1  # TODO: read correctly from metadata

    # [um]
    length_x = shape_x * scale_x
    length_y = shape_y * scale_y
    length_z = shape_z * scale_z
    length_t = shape_t * scale_t

    if "P" in nd2file.sizes:
        loops = {experiment.type: experiment for experiment in nd2file.experiment}
        if "XYPosLoop" not in loops.keys():
            raise ValueError(
                f"The nd2 file {nd2file.path} contains multiple positions, "
                "but no XYPosLoop was found in metadata."
            )
        for p, pnt in enumerate(loops["XYPosLoop"].parameters.points):
            top_l = Point(
                x=pnt.stagePositionUm.x,
                y=pnt.stagePositionUm.y,
                z=pnt.stagePositionUm.z,
                c=0,
                t=0,
            )
            diag = Vector(x=length_x, y=length_y, z=length_z, c=shape_c, t=length_t)
            tile_loader = nd2TileLoader(path=nd2file.path, p=p)
            pixel_size = PixelSize(x=scale_x, y=scale_y, z=scale_z)
            tile = Tile(
                top_l=top_l,
                diag=diag,
                pixel_size=pixel_size,
                data_loader=tile_loader,
            )
            yield tile
    else:
        # TODO: check if this holds...
        pnt = nd2file.frame_metadata(0).channels[0].position
        top_l = Point(
            x=pnt.stagePositionUm.x,
            y=pnt.stagePositionUm.y,
            z=pnt.stagePositionUm.z,
            c=0,
            t=0,
        )
        diag = Vector(x=length_x, y=length_y, z=length_z, c=shape_c, t=length_t)
        pixel_size = PixelSize(x=scale_x, y=scale_y, z=scale_z)
        tile_loader = nd2TileLoader(path=nd2file.path, p=None)
        tile = Tile(
            top_l=top_l,
            diag=diag,
            pixel_size=pixel_size,
            data_loader=tile_loader,
        )
        yield tile

In [288]:
def parse_well_info(fn):
    pattern = r"Well([A-Z])(\d+)"
    match = re.search(pattern, Path(fn).stem)
    if match:
        row = match.group(1)
        col = int(match.group(2))
        return row, col
    else:
        raise ValueError(f"Well info not found in filename {fn}")

In [289]:
def build_tiled_image(
    nd2_path: str | Path,
    zarr_name: str | None = None,
    acquisition_id: int | None = None,
    plate: bool = False,
) -> list[TiledImage]:
    """Build tiled image from nd2 file."""
    nd2file = nd2.ND2File(nd2_path)
    if zarr_name is None:
        _zarr_name = f"{Path(nd2_path).stem}"
        _zarr_name = _zarr_name.replace(" ", "_")
    else:
        _zarr_name = zarr_name

    # load channel info
    channel_names = []
    channel_wavelengths = []
    for channel in nd2file.metadata.channels:
        channel_names.append(channel.channel.name)
        channel_wavelengths.append(channel.channel.excitationLambdaNm)

    # Define path builder for relative ome-zarr path
    if plate:
        row, col = parse_well_info(nd2_path)
        _path_builder = PlatePathBuilder(
            plate_name=_zarr_name,
            row=row,
            column=col,
            acquisition_id=acquisition_id if acquisition_id is not None else 0,
        )
    else:
        if acquisition_id is not None:
            raise NotImplementedError(
                "acquisition_id is not yet supported for non-plate data."
            )
            # TODO: add support for acquisition_id
            # (needs to be implemented in SimplePathBuilder in fractal_converters_tools)
        _path_builder = SimplePathBuilder(
            path=_zarr_name,
        )

    # Build tiles
    tiled_image = TiledImage(
        name=nd2_path,
        path_builder=_path_builder,
        channel_names=channel_names,
        wavelength_ids =channel_wavelengths,
    )
    for tile in build_tiles(nd2file):
        tiled_image.add_tile(tile)

    nd2file.close()
    return tiled_image

In [291]:
out = build_tiled_image(
    nd2_path = fn,
    zarr_name = 'test',
    acquisition_id = None,
    plate = False,
)

In [292]:
fn

'Z:\\zmbstaff\\9987\\Raw_Data\\BVC_converters\\testData_Nikon_ti2\\one channel.nd2'

In [294]:
out.tiles

[Tile(xyzct=(14720.2, -4870.8, 4688.76, 0, 0), diag=(746.1732227916305, 746.1732227916305, 1.0, 1, 1)), origin=(14720.2, -4870.8), space=TileSpace.REAL)]

In [300]:
f"{"test"}_{Path(fn).stem}"

'test_one channel'

In [168]:
for i in build_tiles(nd2file):
    print(i.load().shape)

(1, 2, 1, 512, 1024)


In [105]:
fn

'Z:\\zmbstaff\\9987\\Raw_Data\\BVC_converters\\testData_NikonSD\\ND_Acquisitions\\01_0c_0z.nd2'

In [102]:
nd2file = nd2.ND2File(fn)

shape_x = nd2file.sizes["X"] if "X" in nd2file.sizes else 1
shape_y = nd2file.sizes["Y"] if "Y" in nd2file.sizes else 1
shape_z = nd2file.sizes["Z"] if "Z" in nd2file.sizes else 1
shape_c = nd2file.sizes["C"] if "C" in nd2file.sizes else 1
shape_t = nd2file.sizes["T"] if "T" in nd2file.sizes else 1

# scale factors [um]/[px]
scale_x = nd2file.voxel_size().x
scale_y = nd2file.voxel_size().y
scale_z = nd2file.voxel_size().z
scale_t = 1  # TODO: read correctly from metadata

# [um]
length_x = shape_x * scale_x
length_y = shape_y * scale_y
length_z = shape_z * scale_z
length_t = shape_t * scale_t

for experiment in nd2file.experiment:
    if experiment.type == "XYPosLoop":
        for p, pnt in enumerate(experiment.parameters.points):
            top_l = Point(
                x=pnt.stagePositionUm.x,
                y=pnt.stagePositionUm.y,
                z=pnt.stagePositionUm.z,
                c=0,
                t=0,
            )
            diag = Vector(x=length_x, y=length_y, z=length_z, c=shape_c, t=length_t)

            tile_loader = nd2TileLoader(path=fn, p=p)


C:\Users\local_f.sturzenegger\Temp\23\ipykernel_19964\3175704724.py:1: UserWarning: ND2File file not closed before garbage collection. Please use `with ND2File(...):` context or call `.close()`.
  nd2file = nd2.ND2File(fn)


In [63]:
top_l

Point(x=39523.3, y=-22161.300000000003, z=5720.240000000001, c=0, t=0)